In [ ]:
import json
from datamodel import DataModel

#user evaluation
json_file_path = '/asdf.json'

with open(json_file_path) as f:
    data = json.load(f)

data_model = DataModel.parse_obj(data)

df = data_model.tabular_df()
df.insert(0, 'id', range(1, 1 + len(df)))
data_cols = ['metrics_pass','user_pass',
# 'similarity',
'context-relevance',
'context-faithfulness',
'Factuality',
'Tone',
'answer-relevance']

THRESHOLD = 0.6

# add a calculated score column
metrics_cols = [
    # 'similarity',
    'context-relevance',
    'context-faithfulness',
    'Factuality',
    'Tone',
    'answer-relevance']
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and THRESHOLD is defined
THRESHOLD = 0.6  # Example threshold, adjust as needed

# Define the calculation as a function
def calculate_score(row):
    calced_score = 1  # Start assuming the score is 1
    for col in metrics_cols:
        # Check if the column value exists and is below the threshold
        if pd.notnull(row[col]) and row[col] < THRESHOLD:
            calced_score = 0
            break  # Exit loop early if any score is below threshold
    return calced_score

# Apply the function to each row
df['metrics_pass'] = df.apply(calculate_score, axis=1)

# rename column
df['user_pass'] = df['pass_field']
df = df.drop(columns=['pass_field'])

df['context_len'] = df['context'].apply(lambda x: len(str(x)))
df['context_len']

# rename column
df['ground_truth'] = df['value']
df = df.drop(columns=['value'])

# reorder the df
df = df[['id','query','text','ground_truth','context','comment','Tone','context-relevance','answer-relevance','context-faithfulness','Factuality',
        #  'similarity',
         'metrics_pass','user_pass','context_len']]

df.head(5)

In [ ]:
# get all rows where context relevance is 0
df[df['context-relevance'] == 0]


In [ ]:
df[df['user_pass'] == 0]


In [ ]:
# create a confusion matrix
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

# create a confusion matrix

y_true = df['user_pass']
y_pred = df['context-relevance'] > 0
cm = confusion_matrix(y_pred, y_true)

#plot cm
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt="d")
plt.xlabel('Predicted - Promptfoo')
plt.ylabel('Actual - User evaluation')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
# get all false negatives 
df[(df['user_pass'] == 1) & (df['context-relevance'] == 0)]


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix

# Assuming 'df' is your DataFrame and it has been properly defined with 'user_pass' and 'context-relevance' columns




# Create a confusion matrix
y_true = df['user_pass']
y_pred = df['context-relevance'] > 0
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)  # Adjust to appropriate scale for your paper
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', cbar=False)  # 'Blues' or 'Greys' for monochromatic color
plt.xlabel('Predicted - Promptfoo', fontsize=14)
plt.ylabel('Actual - User Evaluation', fontsize=14)
plt.title('Confusion Matrix of Context Relevance Prediction', fontsize=16)
plt.xticks([0.5, 1.5], ['Negative', 'Positive'])  # Assuming binary classification: 0-Negative, 1-Positive
plt.yticks([0.5, 1.5], ['Negative', 'Positive'], va='center')
plt.tight_layout()  # Adjust layout to make room for label
plt.show()

# accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.2f}')
# f1 
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred)
print(f'F1 Score: {f1:.2f}')


In [ ]:
# do an ROC 
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

y_true = df['user_pass']


fpr, tpr, thresholds = roc_curve(y_true, df['context-relevance'])
roc_auc = roc_auc_score(y_true, df['context-relevance'])

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for context-relevance')
plt.legend(loc='lower right')
plt.show()

# calcualate yudens index
yudens = tpr - fpr
optimal_idx = np.argmax(yudens)
print(f'Optimal Threshold: {thresholds[optimal_idx]}')

# Create a confusion matrix
y_true = df['user_pass']

# use the threshold to predict
y_pred = [1 if e >= thresholds[optimal_idx] else 0 for e in df['context-relevance']]
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)  # Adjust to appropriate scale for your paper
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', cbar=False)  # 'Blues' or 'Greys' for monochromatic color
plt.xlabel('Predicted - Promptfoo', fontsize=14)
plt.ylabel('Actual - User Evaluation', fontsize=14)
plt.title('Confusion Matrix of context-relevance Prediction', fontsize=16)
plt.xticks([0.5, 1.5], ['Negative', 'Positive'])  # Assuming binary classification: 0-Negative, 1-Positive
plt.yticks([0.5, 1.5], ['Negative', 'Positive'], va='center')
plt.tight_layout()  # Adjust layout to make room for label
plt.show()

# accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.2f}')
# f1 
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred)
print(f'F1 Score: {f1:.2f}')


In [ ]:
# do an ROC 
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

y_true = df['user_pass']


fpr, tpr, thresholds = roc_curve(y_true, df['context-faithfulness'])
roc_auc = roc_auc_score(y_true, df['context-faithfulness'])

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for context-faithfulness')
plt.legend(loc='lower right')
plt.show()

# calcualate yudens index
yudens = tpr - fpr
optimal_idx = np.argmax(yudens)
print(f'Optimal Threshold: {thresholds[optimal_idx]}')

# Create a confusion matrix
y_true = df['user_pass']

# use the threshold to predict
y_pred = [1 if e >= thresholds[optimal_idx] else 0 for e in df['context-faithfulness']]
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)  # Adjust to appropriate scale for your paper
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', cbar=False)  # 'Blues' or 'Greys' for monochromatic color
plt.xlabel('Predicted - Promptfoo', fontsize=14)
plt.ylabel('Actual - User Evaluation', fontsize=14)
plt.title('Confusion Matrix of context-faithfulness Prediction', fontsize=16)
plt.xticks([0.5, 1.5], ['Negative', 'Positive'])  # Assuming binary classification: 0-Negative, 1-Positive
plt.yticks([0.5, 1.5], ['Negative', 'Positive'], va='center')
plt.tight_layout()  # Adjust layout to make room for label
plt.show()

# accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.2f}')
# f1 
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred)
print(f'F1 Score: {f1:.2f}')


In [ ]:
# calculate the f1 score
from sklearn.metrics import f1_score
f1_score(y_true, y_pred)


In [ ]:
# create a matrix of distribution plots for each column
import seaborn as sns

# create a matrix of distribution plots for each column
# sns.pairplot(df[[
#     # 'similarity',
#     'context-relevance','context-faithfulness', 'answer-relevance']])

# create a distribution plot for each column 
sns.displot(df['context-relevance'], kde=True)
sns.displot(df['context-faithfulness'], kde=True)
sns.displot(df['answer-relevance'], kde=True)

# summary statistics
df.describe()

In [ ]:
# all cols where context faithfulness is !=1
df[df['context-faithfulness'] != 1]

# count how many rows have context faithfulness != 1
print(f"Number of rows where context relevance is not 1: {len(df[df['context-relevance'] != 1])}")
print(f"contex_relevance accuracy: {round(1-len(df[df['context-relevance'] != 1])/len(df['context-relevance']),2)}")
print(f"Number of rows where context faithfulness is not 1: {len(df[df['context-faithfulness'] != 1])}")
print(f"context-faithfulness accuracy: {round(1-len(df[df['context-faithfulness'] != 1])/len(df['context-faithfulness']),2)}")



In [ ]:
# create a matrix of distribution plots for each column
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# create a matrix of distribution plots for each column
# treat the context-faithfulness column as a binary indicator
df['context-faithfulness_binary'] = df['context-faithfulness'].apply(lambda x: 1 if x == 1 else 0)

# sns.pairplot(df[['context-relevance','context-faithfulness_binary','user_pass']])

# create a confusion matrix for context faithfulness binary and user pass 
y_true = df['user_pass']
y_pred = df['context-faithfulness_binary']
cm = confusion_matrix(y_true, y_pred)

# plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)  # Adjust to appropriate scale for your paper
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', cbar=False)  # 'Blues' or 'Greys' for monochromatic color
plt.xlabel('Predicted - Promptfoo', fontsize=14)
plt.ylabel('Actual - User Evaluation', fontsize=14)
plt.title('Confusion Matrix of Context Faithfulness Prediction', fontsize=16)
plt.xticks([0.5, 1.5], ['Negative', 'Positive'])  # Assuming binary classification: 0-Negative, 1-Positive
plt.yticks([0.5, 1.5], ['Negative', 'Positive'], va='center')
plt.tight_layout()  # Adjust layout to make room for label
plt.show()

# calculate the f1 score
from sklearn.metrics import f1_score
print(f1_score(y_true, y_pred))
#print accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))



In [ ]:
# go through each column and calculate the average, excluding nan values
colnames = []
averages = []
for col in data_cols:
    avg = round(df[col].mean(),2)
    averages.append(avg)
    colnames.append(col)
    print(f'Average {col}: {avg}')

import matplotlib.pyplot as plt

plt.figure(figsize=(18,6))
plt.bar(colnames, averages)
plt.xlabel('Column Name')
plt.ylabel('Average')
plt.title('Average by Column')

# add a line at y=0.6 going through the whole graph
plt.axhline(y=0.6, color='r', linestyle='--')


In [ ]:
# go through each column and calculate the average, excluding nan values
colnames = []
averages = []
df_user_failures = df[df['user_pass'] == False]
for col in data_cols:
    avg = round(df_user_failures[col].mean(),2)
    averages.append(avg)
    colnames.append(col)
    print(f'Average {col}: {avg}')

import matplotlib.pyplot as plt

plt.figure(figsize=(18,6))
plt.bar(colnames, averages)
plt.xlabel('Column Name')
plt.ylabel('Average')
plt.title('Average by Column for failing questions due to user evaluation')

# add a line at y=0.6 going through the whole graph
plt.axhline(y=0.6, color='r', linestyle='--')


In [ ]:
print(df[df['user_pass'] == False])

In [ ]:
df_metric_failures = df[df['metrics_pass'] == 1]

df_different = df_metric_failures[df_metric_failures['user_pass'] == False]

df_different
# df_metric_failures

In [ ]:
# go through each column and calculate the average, excluding nan values
colnames = []
averages = []
df_metric_failures = df[df['metrics_pass'] == 0]
for col in data_cols:
    avg = round(df_metric_failures[col].mean(),2)
    averages.append(avg)
    colnames.append(col)
    print(f'Average {col}: {avg}')

import matplotlib.pyplot as plt

plt.figure(figsize=(18,6))
plt.bar(colnames, averages)
plt.xlabel('Column Name')
plt.ylabel('Average')
plt.title('Average by Column for failing questions due to metrics evaluation')

# add a line at y=0.6 going through the whole graph
plt.axhline(y=0.6, color='r', linestyle='--')


In [ ]:
df['metrics_pass'].value_counts()

In [ ]:
# create a confusion matrix
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix

# create a confusion matrix

y_true = df['user_pass']
y_pred = df['metrics_pass'] > 0
cm = confusion_matrix(y_true, y_pred)

#plot cm
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt="d")
plt.xlabel('Predicted - Promptfoo')
plt.ylabel('Actual - User evaluation')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df_user_failures = df[df['user_pass'] == False]
df_metric_failures = df[df['metrics_pass'] == 0]
#make a boxplot, make sure there is space for the labels
plt.figure(figsize=(18,6))
plt.title('Boxplot of explainers for failing questions due to user eval')
#plot the boxplot, excluding the id column and outliers
sns.boxplot(data=df_user_failures[data_cols])

In [ ]:
mean_dfs =df[data_cols].mean()
mean_dfs.plot(kind='bar', x='query', y=['similarity','context-relevance','context-faithfulness','Factuality','Tone','answer-relevance'])

In [ ]:
# plot similarity and index value as a bar plot
df.plot(kind='bar', x='query', y=[
    # 'similarity',
    'context-relevance','context-faithfulness','Factuality','Tone','answer-relevance'])

In [ ]:
# make a heatmap of the df
plt.figure(figsize=(18,6))
sns.heatmap(df[data_cols].corr(), annot=True)


In [ ]:
data_cols

In [ ]:
# create a matrix of distribution plots for each column
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# create a matrix of distribution plots for each column
sns.pairplot(df[[
    # 'similarity',
    'context-relevance','context-faithfulness','Factuality','Tone', 'answer-relevance']])

In [ ]:
# create a matrix of distribution plots for each column
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# create a matrix of distribution plots for each column
sns.pairplot(df_user_failures[data_cols])


In [ ]:
# df to csv 
df.to_csv('/asdf.csv', index=False,sep=";")



In [ ]:
# df[df['user_pass'] == False]

df_user_failures